# UKG to CMiC Integration Middleware
This python project pulls time entry data from UKG Time Management (UTM) along with associated employee information, transforms it into the necessary format for upload to CMiC pytimesheets, and posts the entries. Check log files for errors.

In [ ]:
import pandas as pd
from datetime import datetime
from pay_period import find_period_by_date,payRuntoPayGroup

selectionDate = datetime(2026,1,9)
#B or W for biweekly or weekly payruns
payRun = 'B'
PAYGROUPID = payRuntoPayGroup[payRun]
pay_period_file = "DataFiles/Pay_Period.csv"
df = pd.read_csv(pay_period_file, skiprows=2)
period_df = find_period_by_date(df,payRun,selectionDate)
period_df

In [ ]:
startdate = period_df['Start Date'].iloc[0]
enddate = period_df['End Date'].iloc[0]
period_control = enddate + pd.Timedelta(days=6)
print(f"{startdate=}, {enddate=},{period_control=}, {payRun=}, {PAYGROUPID=}")

In [ ]:
import ukg_charge_rate

charge_rate_df = ukg_charge_rate.main()

In [ ]:
import utm_load
utm_load.load_ukg(startdate, enddate, PAYGROUPID, charge_rate_df)

from cmic import load_cmic_projects
load_cmic_projects()

In [ ]:
from cmic import employee_push
startdate_str = startdate.strftime("%Y-%m-%d")
employee_push(startdate_str)

In [ ]:
from cmic import post_timesheets_to_CMiC
import pandas as pd
import matplotlib as mpl
df_timesheet_post_results = post_timesheets_to_CMiC(payRun,testing=False)

In [ ]:
df_timesheet_post_results.describe(include='all')

In [ ]:
#since PostResults can contain multiple retries, filter to only most succesful attempt
filtered_post_results = df_timesheet_post_results.groupby(['Time_Id'])['Status'].idxmin()
df_filtered_post_results = df_timesheet_post_results.loc[filtered_post_results]
df_filtered_post_results

In [ ]:
plot_data = df_filtered_post_results.pivot_table(index='WorkDate',columns='Status',values='Hours',aggfunc='count')
chart = plot_data.plot(kind='bar')\
        .legend(loc=(1.05,0.5))\
        .set_title('POST Responses')
chart

In [ ]:
df_filtered_post_results.loc[df_filtered_post_results["Status"]==400]

In [ ]:
df_filtered_post_results.loc[df_timesheet_post_results["Status"]==201].count()